In [3]:
print("Cloud Computing!")

Cloud Computing!


In [4]:
import nltk
import random
from nltk.corpus import twitter_samples
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.classify import NaiveBayesClassifier
from nltk.classify.util import accuracy as nltk_accuracy

# Download the NLTK data
nltk.download('twitter_samples')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')


In [8]:
import re
import string
stopwords_english = stopwords.words('english')
stemmer = PorterStemmer()

def preprocess(tweet):
    # Remove stock market tickers like $GE
    tweet = re.sub(r'\$\w+', '', tweet)

    # Remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)

    # Remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)

    # Remove hashtags (only removing the hash # sign from the word)
    tweet = re.sub(r'#', '', tweet)

    # Tokenize the tweet
    tweet_tokens = word_tokenize(tweet)

    # Remove stopwords and stemming
    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and word not in string.punctuation):
            stem_word = stemmer.stem(word)
            tweets_clean.append(stem_word)

    return tweets_clean

# Preprocess the positive and negative tweets
positive_tweets_clean = []
negative_tweets_clean = []

for tweet in positive_tweets:
    positive_tweets_clean.append(preprocess(tweet))

for tweet in negative_tweets:
    negative_tweets_clean.append(preprocess(tweet))


In [9]:
def get_tweets_for_model(cleaned_tweets):
    for tweet_tokens in cleaned_tweets:
        yield dict([token, True] for token in tweet_tokens)

positive_tweets_model = get_tweets_for_model(positive_tweets_clean)
negative_tweets_model = get_tweets_for_model(negative_tweets_clean)

# Split the dataset into train and test sets
positive_dataset = [(tweet_dict, 'Positive') for tweet_dict in positive_tweets_model]
negative_dataset = [(tweet_dict, 'Negative') for tweet_dict in negative_tweets_model]

dataset = positive_dataset + negative_dataset
random.shuffle(dataset)

train_data = dataset[:7000]
test_data = dataset[7000:]


In [10]:
# Train the classifier
classifier = NaiveBayesClassifier.train(train_data)

# Evaluate the classifier
accuracy = nltk_accuracy(classifier, test_data)
print("Accuracy:", accuracy)


Accuracy: 0.7653333333333333


In [12]:
def classify_tweet(tweet):
    tweet_tokens = preprocess(tweet)
    tweet_dict = dict([token, True] for token in tweet_tokens)
    return classifier.classify(tweet_dict)

# Example usage:
tweet = "I love this movie!"
print(classify_tweet(tweet))


Negative
